<a href="https://colab.research.google.com/github/RPGraciotti/BootCampAlura/blob/main/Modulo_5/Aula_4_valida%C3%A7%C3%A3o_cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

dados = pd.read_excel("https://github.com/alura-cursos/covid-19-clinical/blob/main/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true")
### também posso usar o link no meu repositório

def preenche_tabela(dados):
    features_continuas_colunas = dados.iloc[:, 13:-2].columns
    features_continuas = dados.groupby("PATIENT_VISIT_IDENTIFIER", as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
    features_categoricas = dados.iloc[:, :13]
    saida = dados.iloc[:, -2:]
    dados_finais = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
    dados_finais.columns = dados.columns
    return dados_finais

dados_limpos = preenche_tabela(dados)
a_remover = dados_limpos.query("WINDOW=='0-2' and ICU==1")['PATIENT_VISIT_IDENTIFIER'].values
dados_limpos = dados_limpos.query("PATIENT_VISIT_IDENTIFIER not in @a_remover")
dados_limpos = dados_limpos.dropna()

def prepare_window(rows):
    if(np.any(rows["ICU"])):
        rows.loc[rows["WINDOW"]=="0-2", "ICU"] = 1
    return rows.loc[rows["WINDOW"] == "0-2"]

dados_limpos = dados_limpos.groupby("PATIENT_VISIT_IDENTIFIER").apply(prepare_window)
dados_limpos.AGE_PERCENTIL = dados_limpos.AGE_PERCENTIL.astype("category").cat.codes # transformando em variaveis categoricas


np.random.seed(73246)

x_columns = dados.columns
y = dados_limpos["ICU"]
x = dados_limpos[x_columns].drop(["ICU","WINDOW"], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)
modelo = DummyClassifier()
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


0.48863636363636365

In [8]:
modelo = LogisticRegression(max_iter=10000)
modelo.fit(x_train, y_train)

y_prediction = modelo.predict(x_test)
accuracy_score(y_test, y_prediction)

0.6818181818181818

In [9]:
modelo_arvore = DecisionTreeClassifier()

modelo_arvore.fit(x_train, y_train)
pred_arvore = modelo_arvore.predict(x_test)
accuracy_score(y_test, pred_arvore)


0.6590909090909091

In [10]:

def roda_n_modelos(modelo, dados, n):

  x_columns = dados.columns
  y = dados["ICU"]
  x = dados[x_columns].drop(["ICU","WINDOW"], axis=1)
  auc_lista = []

  for _ in range(n):

    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y)
    modelo.fit(x_train, y_train)
    prob_predic = modelo.predict_proba(x_test)
    auc = roc_auc_score(y_test, prob_predic[:,1])
    auc_lista.append(auc)

  auc_medio = np.mean(auc_lista)
  auc_std = np.std(auc_lista)
  print(f"AUC {auc_medio}")
  print(f"Intervalo {auc_medio - 2 * auc_std} - {auc_medio + 2 * auc_std}") # confidence interval
  #plt.hist(auc_lista)

# Aula 4

In [11]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

In [12]:
cv = StratifiedKFold(n_splits = 5, shuffle = True)
cross_validate(modelo, x, y, cv = cv) # comumente utilizado entre 5 e 10, tambem existe um tradeoff ai entre tamanho do dataset

{'fit_time': array([0.66037488, 0.2530818 , 0.21855068, 0.5462029 , 0.20197511]),
 'score_time': array([0.0036273 , 0.00440955, 0.00242567, 0.00244927, 0.00230646]),
 'test_score': array([0.70422535, 0.74647887, 0.74285714, 0.65714286, 0.75714286])}

In [13]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [14]:
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 10)
cross_validate(modelo, x, y, cv = cv)

{'fit_time': array([0.64826941, 0.58294034, 0.57252741, 0.24404216, 0.62090445,
        0.53883624, 0.51787114, 0.54570365, 0.21226597, 0.19857407,
        0.25331736, 0.72981644, 0.54429388, 0.66699934, 0.49113655,
        0.62171793, 0.30299067, 0.64509439, 0.75889134, 0.57082415,
        0.55788898, 0.23813367, 0.1566546 , 0.64110136, 0.23812103,
        0.57019949, 0.54462051, 0.65144587, 0.26358342, 0.59403324,
        0.63438559, 0.48990035, 0.24571681, 0.34674835, 0.59028172,
        0.61471415, 0.25450921, 0.57914233, 0.71438909, 0.52601099,
        0.56006789, 0.68821812, 0.57620096, 0.62767577, 0.65170574,
        0.57064438, 0.58705139, 0.56636429, 0.60725498, 0.30100703]),
 'score_time': array([0.00232911, 0.00229096, 0.00229478, 0.00230527, 0.00301528,
        0.00224113, 0.00226474, 0.00260663, 0.00252819, 0.00399065,
        0.00269651, 0.00244975, 0.00682759, 0.00228763, 0.00223899,
        0.00236702, 0.00259471, 0.00251985, 0.00227451, 0.0022893 ,
        0.00257349, 

In [15]:
def roda_modelo_cv(modelo, dados, n_splits, n_repeats):

    np.random.seed(1231234) # lembrando sempre que é para valor didático
    dados = dados.sample(frac = 1).reset_index(drop = True) # embaralha
    x_columns = dados.columns
    y = dados["ICU"]
    x = dados[x_columns].drop(["ICU","WINDOW"], axis=1)

    cv = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats = n_repeats)
    resultados = cross_validate(modelo, x, y, cv = cv, scoring = "roc_auc")

    auc_medio = np.mean(resultados["test_score"])
    auc_std = np.std(resultados["test_score"])

    print(f"AUC Médio {auc_medio}") 
    print(f"Intervalo {auc_medio - (2 * auc_std)} - {auc_medio + (2 * auc_std)}")


In [16]:
roda_modelo_cv(modelo, dados_limpos, 5, 10)

AUC Médio 0.7575511875511876
Intervalo 0.6366273412799195 - 0.8784750338224556


In [17]:
roda_n_modelos(modelo, dados_limpos, 50)

AUC 0.7611728074727554
Intervalo 0.6552383213129391 - 0.8671072936325717


# Desafio 01 - Testar outros splitter classes e observar as diferenças

https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py

Testarei um outro de tipo de algoritmo, de embaralhamento "completo", ou "shuffle", e assim como na aula, também é possível estratificá-lo, a fim de garantir que tenhamos sempre quantidades próximas de 0s e 1s nos nossos dados de treino e dados de teste.

In [23]:
from sklearn.model_selection import (ShuffleSplit,StratifiedShuffleSplit)

In [25]:
# Shuffle Split
def roda_modelo_cv(modelo, dados, n_splits):

    np.random.seed(1231234) # lembrando sempre que é para valor didático
    dados = dados.sample(frac = 1).reset_index(drop = True) # embaralha
    x_columns = dados.columns
    y = dados["ICU"]
    x = dados[x_columns].drop(["ICU","WINDOW"], axis=1)

    cv = ShuffleSplit(n_splits = n_splits)
    resultados = cross_validate(modelo, x, y, cv = cv, scoring = "roc_auc")

    auc_medio = np.mean(resultados["test_score"])
    auc_std = np.std(resultados["test_score"])

    print(f"AUC Médio {auc_medio}") 
    print(f"Intervalo {auc_medio - (2 * auc_std)} - {auc_medio + (2 * auc_std)}")

roda_modelo_cv(modelo_arvore, dados_limpos, 5)

AUC Médio 0.623299425033171
Intervalo 0.505045201056298 - 0.7415536490100441


In [26]:
# Stratified Shuffle Split
def roda_modelo_cv(modelo, dados, n_splits):

    np.random.seed(1231234) # lembrando sempre que é para valor didático
    dados = dados.sample(frac = 1).reset_index(drop = True) # embaralha
    x_columns = dados.columns
    y = dados["ICU"]
    x = dados[x_columns].drop(["ICU","WINDOW"], axis=1)

    cv = StratifiedShuffleSplit(n_splits = n_splits)
    resultados = cross_validate(modelo, x, y, cv = cv, scoring = "roc_auc")

    auc_medio = np.mean(resultados["test_score"])
    auc_std = np.std(resultados["test_score"])

    print(f"AUC Médio {auc_medio}") 
    print(f"Intervalo {auc_medio - (2 * auc_std)} - {auc_medio + (2 * auc_std)}")

roda_modelo_cv(modelo_arvore, dados_limpos, 5)

AUC Médio 0.6597523219814242
Intervalo 0.4902760248196175 - 0.8292286191432309


Ambos métodos de "shuffling" parecem resultar em uma variância muito grande da acurácia, indo de 49 ~ 82%. Definitivamente não parece representar nenhum ganho de performance em relação aos outros métodos que utilizamos. Considero que esse passo é de suma importância na correta preparação para treino do modelo, e cabe ressaltar que uma análise aprofundada caso a caso se faz necessária quando escolher o melhor método de separar dados de treino e de teste.